In [27]:
import numpy as np

matA = np.array([[1,0,0],[-3,1,0],[0,0,1]])
matB = np.array([[1,2,1],[3,8,1],[0,4,1]])


# Matrix Multiplication

In [28]:
#The numpy easy way
print(np.dot(matA, matB))


[[ 1  2  1]
 [ 0  2 -2]
 [ 0  4  1]]


In [32]:
#reinventing the wheel
def my_matrix_multiply(A, B):
    '''Multiply two matrices'''
    numRowA = len(A)
    numColA = len(A[0])
    numRowB = len(B)
    numColB = len(B[0])   
    
    #requirement for multiplication is the number of columns of A must match the number of rows of B
    if numColA != numRowB:
        print("Error multiplying: Columns of A do not equal rows of B")
        return
    
    #If A is m by n and B is n by p then C will be m by p. 
    #or, basically, the number of rows from A and the number of columns from B
    C = np.zeros(shape=(numRowA, numColB))
    
    #Then, any C[i][j] will be the sum of the row i from A and col j from B
    for i in range(numRowA):
        for j in range(numColB):
            for k in range(numColA):
                C[i][j] += A[i][k] * B[k][j]
    return C

print(my_matrix_multiply(matA, matB))


[[ 1.  2.  1.]
 [ 0.  2. -2.]
 [ 0.  4.  1.]]


# Identity Matrix

In [33]:
print( np.identity(3))

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [37]:
def my_identity(size):
    '''generate identity matrix of parameter size'''
    C = np.zeros(shape=(size, size))
    for row in range(size):
        C[row][row]=1
    return C
print( my_identity(4))

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


# Linear Equations
This solves a system of equations (n equations in n unknowns) of Ax = b where: 
* A is the coefficient matrix of size n by n 
* x is an unknown column vector 
* b is a known column vector. 

This is pretty tough to understand so it is heavily sprinkled with comments and debugging prints because the goal is to learn what is happening. 

Also, super inefficient, so don't really use this. The numpy version is way better since it uses a different technique. This code uses gaussian elimination with back substitution, the numpy solver uses LU-decomposition.

I leaned heavily on this: https://martin-thoma.com/solving-linear-equations-with-gaussian-elimination/

In [43]:
import numpy as np
def my_linearsolver(A,b):
    '''solve the linear equations '''
    n = len(A)   
    #print ("Rowcount is: {}".format(n))
    #print("Original Matrix is: \n{}".format(A))
    
    #construct the augmented matrix
    #print("Shape of A: {}".format(A.shape))
    widthOfB=0
    if b.ndim == 1:
        widthOfB=1
    else:
        raise ValueError("This function is valid only for b as a column vector")
    M = np.zeros(shape=(n, len(A[0])+widthOfB ))
    for x in range(n):
        M[x] = np.append(A[x], b[x])
    #print("Augmented Matrix is: \n{}".format(M))

    #for each row
    for i in range(n):
        #print("i is: {}".format(i))
        maxEl = abs(M[i][i])
        maxRow = i
        #print("maxEl is: {} and maxRow is: {}".format(maxEl, maxRow))
        #go through the rest of the rows
        for k in range(i+1, n):
            #print("k is: {}".format(k))
            #check if the is bigger than the current max
            if abs(M[k][i]) > maxEl:
                #print("M[{}][{}]={} is greater than maxEl={}".format(k,i,abs(M[k][i]),maxEl))
                maxEl = abs(M[k][i])
                maxRow = k
        #swap the max row with the current row
        #print("Swapping rows {} and {}".format(maxRow, i))
        for k in range(i, n+widthOfB):
            tmp = M[maxRow][k]
            M[maxRow][k] = M[i][k]
            M[i][k] = tmp
        #print("After row swap: \n{}".format(M))

        # Make all rows below this one 0 in current column
        for k in range(i+1, n):
            c = -M[k][i]/M[i][i]
            for j in range(i, n+widthOfB):
                if i == j:
                    M[k][j] = 0
                else:
                    M[k][j] += c * M[i][j]
    #print("Presolved Matrix: \n{}".format(M))

    # Solve equation Ax=b 
    x = np.zeros(shape=(b.shape))
    for i in range(n-1, -1, -1):
        #print("i is: {}, M[{}][{}] is: {}, M[{}][{}] is: {}, so x[{}] becomes {}".format( i, i, n, M[i][n], i, i, M[i][i], i,  M[i][n]/M[i][i] ))
        x[i] = M[i][n]/M[i][i]
        for k in range(i-1, -1, -1):
            #print("k is {}, M[{}][{}] is {}, x[{}] is {} so M[{}][{}] becomes {} - {}".format( k, k,i, M[k][i], i, x[i], k,n, M[k][n], M[k][i] * x[i]     ))
            M[k][n] -= M[k][i] * x[i]
        #print(M)
    return x  

print("------------Testing 3x3-------------")
A = np.array([[3, 2, -4], [2, 3, 3], [5, -3, 1]])
b = np.array([3, 15, 14])
#should yield [3,1,2]
print("My solver says: \n{}".format(my_linearsolver(A,b)))
print("numpy says: \n{}".format(np.linalg.solve(A,b)))


print("------------Testing 3x3-------------")
C = np.array([[1,2,1], [3,8,1], [0,4,1]])
d = np.array([2,12,2])
#should yield [2,1,-2]
print("My solver says: \n{}".format(my_linearsolver(C,d)))
print("numpy says: \n{}".format(np.linalg.solve(C,d)))


print("------------Testing 2x2-------------")
E = np.array([[2,5], [1,3]])
f = np.array([12,7])
#should yield [1,2]
print("My solver says: \n{}".format(my_linearsolver(E,f)))
print("numpy says: \n{}".format(np.linalg.solve(E,f)))

print("-----------A random 20x20--------------")
np.random.seed(42)
E = np.random.rand(20,20)
f = np.random.rand(20)
print("My solver says: \n{}".format(my_linearsolver(E,f)))
print("numpy says: \n{}".format(np.linalg.solve(E,f)))

print("----------------Some timing tests---------------------")
import time
start_time = time.clock()
np.random.seed(42)
for i in range(100):
    E = np.random.rand(100,100)
    f = np.random.rand(100)
    my_linearsolver(E,f)
print("My solver solved 100 100x100 in: {} seconds".format( time.clock() - start_time ))

start_time = time.clock()
np.random.seed(42)
for i in range(100):
    E = np.random.rand(100,100)
    f = np.random.rand(100)
    np.linalg.solve(E,f)
print("numpy solved 100 100x100 in: {} seconds".format( time.clock() - start_time ))



------------Testing 3x3-------------
My solver says: 
[ 3.  1.  2.]
numpy says: 
[ 3.  1.  2.]
------------Testing 3x3-------------
My solver says: 
[ 2.  1. -2.]
numpy says: 
[ 2.  1. -2.]
------------Testing 2x2-------------
My solver says: 
[ 1.  2.]
numpy says: 
[ 1.  2.]
-----------A random 20x20--------------
My solver says: 
[-0.20828059 -0.40664065 -0.36293332 -0.26176512  0.11602893  0.6020817
 -0.02981623  0.74717283  0.48120409  0.26134339  0.48016756 -0.2804258
  0.35941802 -0.77387085  0.26656988 -0.14151787  0.36810365 -0.9524231
  0.42416961  0.23218147]
numpy says: 
[-0.20828059 -0.40664065 -0.36293332 -0.26176512  0.11602893  0.6020817
 -0.02981623  0.74717283  0.48120409  0.26134339  0.48016756 -0.2804258
  0.35941802 -0.77387085  0.26656988 -0.14151787  0.36810365 -0.9524231
  0.42416961  0.23218147]
----------------Some timing tests---------------------
My solver solved 100 100x100 in: 24.980617000000002 seconds
numpy solved 100 100x100 in: 0.13017999999999574 secon

# Matrix Inverse

This should give the inverse of a matrix using the Gauss Jordan method. 
In this method we concatenate the matrix with an equal size identity matrix, then use row operations to reduce the original matrix to the identity, and in doing so will reveal the inverse on the right. 

This isn't checking for the matrix being singular, so expect some failure if your matrix isn't invertible. 

Also, by the timing tests at the end, you can see that this is horribly inefficient compared to numpy, so use this only for learning. 

In [44]:
import numpy as np

def my_inverse(A):
    '''returns the gauss jordan inverse matrix'''
    n = len(A)
    #get an identity matrix to augment with
    I = np.identity(n)
    
    M = np.concatenate((A, I), axis=1)
    #print("Augmented matrix is: \n{}".format(M))
    
    #here we use the gaussian elimination from the linear solver before to zero the lower triangle
    #for each row
    for i in range(n):
        #print("i is: {}".format(i))
        maxEl = abs(M[i][i])
        maxRow = i
        #print("maxEl is: {} and maxRow is: {}".format(maxEl, maxRow))
        #go through the rest of the rows
        for k in range(i+1, n):
            #print("k is: {}".format(k))
            #check if the is bigger than the current max
            if abs(M[k][i]) > maxEl:
                #print("M[{}][{}]={} is greater than maxEl={}".format(k,i,abs(M[k][i]),maxEl))
                maxEl = abs(M[k][i])
                maxRow = k
        #swap the max row with the current row
        #print("Swapping rows {} and {}".format(maxRow, i))
        for k in range(i, n+n):
            tmp = M[maxRow][k]
            M[maxRow][k] = M[i][k]
            M[i][k] = tmp
        #print("After row swap: \n{}".format(M))

        # Make all rows below this one 0 in current column
        for k in range(i+1, n):
            c = -M[k][i]/M[i][i]
            for j in range(i, n+n):
                if i == j:
                    M[k][j] = 0
                else:
                    M[k][j] += c * M[i][j]
    #print("Solved for Upper Triangle: \n{}".format(M))
    
    #backward up the rows, to eliminate the upper triangle
    for i in range(n-1,0, -1):
        #print("Base row {}".format(i))
        # Make all rows above this one 0 in current column
        for k in range(i-1, -1, -1):
            #print("Elim row {}".format(k))
            c = -M[k][i]/M[i][i]
            for j in range(i, n+n):
                if i == j:
                    M[k][j] = 0
                else:
                    M[k][j] += c * M[i][j]
    #print("Upper Triangle also solved: \n{}".format(M))
    
    #again through the rows to divide each row by it's pivot so we have identity in the pivot points
    for i in range(n):
        piv = M[i][i]
        for j in range (i,n+n):
            M[i][j] = M[i][j] / piv
    #print("Reduced Row Echelon: \n{}".format(M))
    
    #construct the return matrix
    retMat = np.zeros(shape=(n, n))
    for i in range(n):
        for j in range(n):
            retMat[i][j]=M[i][j+n]
    return retMat

print("A 2x2")
A = np.array([[1,3], [2, 7]])
print("Numpy says the inverse is: \n{}".format(np.linalg.inv(A)))
print("my_inverse says: \n{}".format(my_inverse(A)))
print("\nA 3x3")
A = np.array([[3, 2, -4], [2, 3, 3], [5, -3, 1]])
print("Numpy says the inverse is: \n{}".format(np.linalg.inv(A)))
print("my_inverse says: \n{}".format(my_inverse(A)))
print("\nA 4x4")
A = np.array([[3, 2, -4, 6], [2, 3, 3, 4], [5, -3, 1, 9],[2,4,-3,7]])
print("Numpy says the inverse is: \n{}".format(np.linalg.inv(A)))
print("my_inverse says: \n{}".format(my_inverse(A)))

print("\nSolve Ax=b with the inverse formula x=inv(A)B")
A = np.array([[3, 2, -4], [2, 3, 3], [5, -3, 1]])
b = np.array([3, 15, 14])
#should yield [3,1,2]
print("numpy says: \n{}".format(np.linalg.solve(A,b)))
print("Inverse method says:\n{}".format( np.dot(my_inverse(A),b)  ))


print("----------------Some timing tests---------------------")
import time
start_time = time.clock()
np.random.seed(42)
for i in range(100):
    E = np.random.rand(100,100)
    my_inverse(E)
print("My inverse inverted 100 100x100 in: {} seconds".format( time.clock() - start_time ))

start_time = time.clock()
np.random.seed(42)
for i in range(100):
    E = np.random.rand(100,100)
    np.linalg.inv(E)
print("numpy inverted 100 100x100 in: {} seconds".format( time.clock() - start_time ))

A 2x2
Numpy says the inverse is: 
[[ 7. -3.]
 [-2.  1.]]
my_inverse says: 
[[ 7. -3.]
 [-2.  1.]]

A 3x3
Numpy says the inverse is: 
[[ 0.08219178  0.06849315  0.12328767]
 [ 0.0890411   0.15753425 -0.11643836]
 [-0.14383562  0.13013699  0.03424658]]
my_inverse says: 
[[ 0.08219178  0.06849315  0.12328767]
 [ 0.0890411   0.15753425 -0.11643836]
 [-0.14383562  0.13013699  0.03424658]]

A 4x4
Numpy says the inverse is: 
[[ 0.74936061  0.33248082 -0.10230179 -0.70076726]
 [ 0.13043478  0.17391304 -0.13043478 -0.04347826]
 [-0.15601023  0.12531969  0.03836317  0.01278772]
 [-0.35549872 -0.14066496  0.1202046   0.37340153]]
my_inverse says: 
[[ 0.74936061  0.33248082 -0.10230179 -0.70076726]
 [ 0.13043478  0.17391304 -0.13043478 -0.04347826]
 [-0.15601023  0.12531969  0.03836317  0.01278772]
 [-0.35549872 -0.14066496  0.1202046   0.37340153]]

Solve Ax=b with the inverse formula x=inv(A)B
numpy says: 
[ 3.  1.  2.]
Inverse method says:
[ 3.  1.  2.]
----------------Some timing tests--------

# A= LU factorization
Here we factor the matrix A into lower and upper triangles, L and U. 
Then you can do the trick of Lc=b and Ux = c to solve for x. 

This is, in theory, computationally more efficient than the previous methods. 

I'm going from the MATLAB code from the book, which is close to http://web.mit.edu/18.06/www/Course-Info/Mfiles/slu.m. I converted to python. 

Here, I am using deepcopy to preserve the original copies for the testing purposes, but if you were worried about computational/memory intensity you wouldn't deepcopy. 

In [65]:
import numpy as np
from copy import deepcopy
def slu(A):
    '''Square LU factorization with no row exchanges'''
    A = deepcopy(A)
    n = len(A)
    tol = 1.0E-6
    L = np.zeros(shape=(n, n))
    U = np.zeros(shape=(n, n))
    for k in range(n):
        #if zero found in pivot, error out
        if np.abs(A[k][k]) < tol:
            raise ValueError("Can't go on without row exchange.")
        L[k][k] = 1 #The L matrix automatically gets 1's in the pivot points.
        for i in range(k+1,n):
            L[i][k] = A[i][k] / A[k][k]
            for j in range(k+1, n):
                A[i][j] = A[i][j] - (L[i][k]*A[k][j])
        for j in range(k, n):
            U[k][j] = A[k][j]
    return (L,U)
                
def slv(A, b):
    '''solves Ax=b using L and U'''
    A = deepcopy(A) 
    b = deepcopy(b)
    n = len(A)
    L,U = slu(A)
    c = np.zeros(shape=(n))
    x = np.zeros(shape=(n))
    s = 0  
    #first loop solves Lc=b
    for k in range(n):
        for j in range(k):
            s = s + L[k][j] * c[j]
        c[k] = b[k] - s
        s = 0
    #second loop solves Ux=b
    for k in range(n-1, -1, -1):
        t = 0
        for j in range(k+1, n):
            t = t + U[k][j]*x[j]
        x[k] = (c[k]-t) / U[k][k]
    return x


print("--------LU Factoring-----------")
A = np.array([[2.,1.,0.], [1.,2.,1.], [0.,1.,2.]])
print("L and U are: \n{}".format(slu(A)))

print("----------------A known 3x3---------------------")
A = np.array([[3., 2., -4.], [2., 3., 3.], [5., -3., 1.]])
b = np.array([3., 15., 14.])
#should yield [3,1,2]
print("My solver says: \n{}".format(slv(A,b)))
print("numpy says: \n{}".format(np.linalg.solve(A,b)))

print("----------------A random 10x10---------------------")
np.random.seed(42)
E = np.random.rand(10,10)
f = np.random.rand(10)
print("My solver says: \n{}".format(slv(E,f)))
print("numpy says: \n{}".format(np.linalg.solve(E,f)))

print("----------------Some timing tests---------------------")
import time
start_time = time.clock()
np.random.seed(42)
for i in range(100):
    E = np.random.rand(100,100)
    f = np.random.rand(100)
    slv(E,f)
print("My solver solved 100 100x100 in: {} seconds".format( time.clock() - start_time ))

start_time = time.clock()
np.random.seed(42)
for i in range(100):
    E = np.random.rand(100,100)
    f = np.random.rand(100)
    np.linalg.solve(E,f)
print("numpy solved 100 100x100 in: {} seconds".format( time.clock() - start_time ))


        

--------LU Factoring-----------
L and U are: 
(array([[ 1.        ,  0.        ,  0.        ],
       [ 0.5       ,  1.        ,  0.        ],
       [ 0.        ,  0.66666667,  1.        ]]), array([[ 2.        ,  1.        ,  0.        ],
       [ 0.        ,  1.5       ,  1.        ],
       [ 0.        ,  0.        ,  1.33333333]]))
----------------A known 3x3---------------------
My solver says: 
[ 3.  1.  2.]
numpy says: 
[ 3.  1.  2.]
----------------A random 10x10---------------------
My solver says: 
[-1.01269542 -1.4383189   1.92334288  1.216233   -2.05633017  0.75820865
  1.64963394 -1.47380018  1.50250928  0.17271319]
numpy says: 
[-1.01269542 -1.4383189   1.92334288  1.216233   -2.05633017  0.75820865
  1.64963394 -1.47380018  1.50250928  0.17271319]
----------------Some timing tests---------------------
My solver solved 100 100x100 in: 33.02124199999997 seconds
numpy solved 100 100x100 in: 0.13359700000000885 seconds


# Matrix Transpose

Swap the rows for the columns.

In [69]:
import numpy
#easy in numpy
A = np.array([[1,3], [2, 7]])
print("The matrix:")
print(A)
print("The transpose:")
print(np.matrix.transpose(A))
A = np.array([[3, 2, -4, 6], [2, 3, 3, 4], [5, -3, 1, 9],[2,4,-3,7]])
print("The matrix:")
print(A)
print("The transpose:")
print(np.matrix.transpose(A))

The matrix:
[[1 3]
 [2 7]]
The transpose:
[[1 2]
 [3 7]]
The matrix:
[[ 3  2 -4  6]
 [ 2  3  3  4]
 [ 5 -3  1  9]
 [ 2  4 -3  7]]
The transpose:
[[ 3  2  5  2]
 [ 2  3 -3  4]
 [-4  3  1 -3]
 [ 6  4  9  7]]


In [70]:
import numpy
def my_transpose(A):
    n = len(A)
    w = len(A[0])
    
    At = np.zeros(shape=(w,n))
    for i in range(n):
        for j in range(w):
            At[i][j] = A[j][i]
    return At

A = np.array([[1,3], [2, 7]])
print("The matrix:")
print(A)
print("The transpose:")
print(my_transpose(A))
A = np.array([[3, 2, -4, 6], [2, 3, 3, 4], [5, -3, 1, 9],[2,4,-3,7]])
print("The matrix:")
print(A)
print("The transpose:")
print(my_transpose(A))

The matrix:
[[1 3]
 [2 7]]
The transpose:
[[ 1.  2.]
 [ 3.  7.]]
The matrix:
[[ 3  2 -4  6]
 [ 2  3  3  4]
 [ 5 -3  1  9]
 [ 2  4 -3  7]]
The transpose:
[[ 3.  2.  5.  2.]
 [ 2.  3. -3.  4.]
 [-4.  3.  1. -3.]
 [ 6.  4.  9.  7.]]


# Matrix symmetry

Check that a matrix is symmetric. This uses the property that symmetric matrices are equal to their transpose. The tolerance stuff is to get around some wiggle in floats. 

In [80]:
import numpy

#do this in numpy
A = np.array([[3, 1, 7], [1, 2, 9], [7, 9, 4]])
print( numpy.allclose(A, A.T, atol=1e-8) )

B = np.array([[3, 1, 7], [1, 2, 9], [7, 7, 4]])
print( numpy.allclose(B, B.T, atol=1e-8) )

True
False
